In [3]:
import math
import json

print("treinamento")

with open('train.json') as f:
    usuarios = json.load(f)
print(len(usuarios))
print("treinamento")

treinamento
133385
treinamento


Calculo da similadiridade

In [8]:
print("start")
lambda_differ = 0.7
for i in range(0,1):
    print('user:',i)
    for j in range(1,5):
        print('user1:',j)
        filmes_i = usuarios[i]["filmes"]
        filmes_j = usuarios[j]["filmes"]
        somatorio_1 = 0
        somatorio_2 = 0
        somatorio_3 = 0
        sumdiffer = 0
        m = 0
        for f1 in filmes_i:
            f2 = list(filter(lambda y: y["id"] == f1["id"],filmes_j))
            if f2:
                f2 = f2[0]
                s1 = (f1["avaliacao"] - usuarios[i]["media"])
                s2 = (f2["avaliacao"] - usuarios[j]["media"])
                somatorio_1 += s1*s2
                somatorio_2 += math.pow(s1, 2)
                somatorio_3 += math.pow(s2, 2)
                m = m + 1
                sumdiffer += math.pow(f1["avaliacao"] - f2["avaliacao"], 2)
        raiz = math.sqrt( somatorio_2 * somatorio_3)
        if (m):
            sumdiffer = math.sqrt( sumdiffer/m)
        else:
            sumdiffer = 0
        if (raiz): #verifica divizao por zero
            similaridade = somatorio_1 / raiz
        else:
            similaridade = 0
        if(m < 50):
            similaridade = similaridade / (50-m)
        similaridade = similaridade *  math.pow(lambda_differ, sumdiffer)
        if(similaridade > 0):
            usuarios[i]["user_similarity"].append((usuarios[j]["id"], similaridade))
            usuarios[j]["user_similarity"].append((usuarios[i]["id"], similaridade))
            
print(usuarios[0]["user_similarity"])
with open('usuarios.json', 'w') as outfile:
    json.dump(usuarios[0], outfile)

start
user: 0
user1: 1
user1: 2
user1: 3
user1: 4
[(885013.0, 0.014285714285714289), (885013.0, 0.014285714285714289), (885013.0, 0.014285714285714289), (885013.0, 0.014285714285714289), (885013.0, 0.014285714285714289), (885013.0, 0.014285714285714289), (885013.0, 0.014285714285714289)]


In [ ]:
import itertools, json
# from IPython.display import clear_output

def find_item_by_id(item_id, data):
    search = list(filter(lambda x: x['id'] == item_id, data))
    if(len(search) == 1):
        return search[0]
    else:
        return None
    
def get_id_users(data):
    return list(map(lambda x: x['id'], data))
    
def get_movies(user_id, data):
    user = find_item_by_id(user_id, data)
    return user['filmes']

def non_watch_movies(user_movies, neighborhood, data):
    all_nb_movies = list(map(lambda x: get_movies(x[0], data), neighborhood)) # all movies seen by the neighborhood
    all_nb_movies = list(itertools.chain.from_iterable(all_nb_movies)) # [[]] -> []
    all_nb_movies = list(set(map(lambda x: x['id'], all_nb_movies))) # IDs of all neighborhood movies
    u_movies = list(map(lambda x: x['id'], user_movies)) # IDs of user movies
    
    return list(filter(lambda x: x not in u_movies, all_nb_movies)) # movies not seen by the user in his neighborhood

def just_famous_movies(neighborhood, movies, n_movies, data): # (neighborhood) nearest neighborhood (e.g. 40 nearest users by similarity)
    nb_ids = list(map(lambda x: x[0], neighborhood)) # id of (id, similarity)
    nb = list(filter(lambda x: x['id'] in nb_ids, data)) # neighbors by id
    filmes_nb = list(map(lambda x: x['filmes'], nb)) # rating of neighbors [[]]
    filmes_nb = list(itertools.chain.from_iterable(filmes_nb)) # [[]] -> []
    
    if len(filmes_nb) == 0:
        return {'famous_movies': []}
    
    filmes_nb = list(map(lambda x: x['id'], filmes_nb)) # IDs of filmes    
    filmes_nb = list(map(lambda x: (x, filmes_nb.count(x)), movies)) # (movie, frequency)
    filmes_nb.sort(key=lambda x: x[1], reverse=True) # sort by highest frequency
    famous_movies = filmes_nb[:n_movies] # n popular movies
    famous_movies = list(map(lambda x: x[0], famous_movies)) # only IDs, frequency is not necessary, just for ranking
    
#     if len(famous_movies) > 0: # hard to happen, but...
        # search neighbors who watched at least one popular movie
    nb_watched_famous_movies = list(filter(lambda x: any(movie in famous_movies for movie in list(map(lambda y: y['id'], x['filmes'])) ), nb))
        
        # return in 'nb' the users obj to ease later, but it can be just IDs
    return {'famous_movies': famous_movies, 'nb': nb_watched_famous_movies}
#     else:
#         print("O valor de n = ", n, " é muito grande. Não existe um filme que ", n, " vizinhos assistiram.")
#         return {}
    
def find_similarity(neighborhood, user_id):
    search = list(filter(lambda x: x[0] == user_id, neighborhood))
    if(len(search) == 1):        
        return search[0][1] # return only similarity value
    else:
        raise Exception('not find')

all_predict = []
unpredictable = []
dataset_test = json.load(open('./test.json'))
def predict_user_movie(user, n, num, data):
#     n = 2 # neighborhood
#     min_movies = 1 # min value of filmes
    
    neighborhood = user['user_similarity']
    neighborhood.sort(key=lambda x: x[1], reverse=True) # sort by similarity
    neighborhood = neighborhood[:n] # the real neighborhood
    
    # movies watched by neighborhood but not by the user
#     non_watched_movies = non_watch_movies(user['filmes'], neighborhood, data)

    # just the movies with high frequency within neighborhood
#     famous_movies = just_famous_movies(neighborhood, non_watched_movies, min_movies, data)
    
#     if famous_movies['famous_movies'] == []:
#         unpredictable.append({'user_id': user['id']})
#     else:
#         nb = famous_movies['nb'] # users
#         famous_movies = famous_movies['famous_movies'] # movies IDs

#         nb_ids = list(map(lambda x: x['id'], nb)) # just IDs
#         neighborhood = list(filter(lambda x: x[0] in nb_ids, neighborhood)) # filter by IDs -> (user_id, similarity)

    pa_is = [] # final prediction (movie_id, prediction_value)
    
    nb = list(map(lambda x: find_item_by_id(x[0],data), neighborhood))
    main_user = find_item_by_id(user['id'], dataset_test)
#     print(main_user['filmes'])
    famous_movies = list(map(lambda x: x['id'], main_user['filmes']))
    
    for movie_id in famous_movies:
#         print("filme:",movie_id)
        sum_1 = 0
        sum_2 = 0
        for i in nb:            
            sim = find_similarity(neighborhood, i['id']) # search in list [(user_id, similarity)]
            mv = find_item_by_id(movie_id, i['filmes']) # retrieve movie in user movies list
            if mv == None:
#                 sum_1 = sum_1 + ((0 - i['media']) * sim)
                sum_2 = sum_2 + sim
#                 print(i['id'], "não assistiu:", movie_id)
            else:
#                 print("user:",i['id'],"filmes:",i['filmes'])
                sum_1 = sum_1 + ((mv['avaliacao'] - i['media']) * sim)
                sum_2 = sum_2 + sim
#                 print(i['id'], "assistiu:", movie_id)            
        if sum_2 > 0:
            pa_i = user['media'] + (sum_1/sum_2)
        else:
            pa_i = user['media']
        pa_is.append({'id':movie_id, 'predicao': pa_i})
    
    with open('predicao_05.json', 'a') as outfile:
        outfile.write(json.dumps({'id': user['id'], 'filmes': pa_is})+",\n")
        outfile.close()
    print("completo:", (((num+1)*100)/80000),"%")
#     all_predict.append({'id': user['id'], 'filmes': pa_is})
    
dataset = json.load(open('./vizinhanca_05.json'))
users = get_id_users(dataset)
# tam = len(users)

with open('predicao_05.json', 'a') as outfile:
    outfile.write("[\n")
    outfile.close()
    
for i, id_user in enumerate(users):
#     clear_output()
#     print("Restam:", tam-1)
#     tam = tam-1
    user = find_item_by_id(id_user, dataset)
    predict_user_movie(user, 50, i, dataset)
    
with open('predicao_05.json', 'a') as outfile:
    outfile.write("]")
    outfile.close()
# all_predict

# with open('predictable.json', 'w') as outfile:
#     json.dump(all_predict, outfile)
    
# with open('unpredictable.json', 'w') as outfile:
#     json.dump(unpredictable, outfile)

completo: 0.00125 %
completo: 0.0025 %
completo: 0.00375 %
completo: 0.005 %
completo: 0.00625 %
completo: 0.0075 %
completo: 0.00875 %
completo: 0.01 %
completo: 0.01125 %
completo: 0.0125 %
completo: 0.01375 %
completo: 0.015 %
completo: 0.01625 %
completo: 0.0175 %
completo: 0.01875 %
completo: 0.02 %
completo: 0.02125 %
completo: 0.0225 %
completo: 0.02375 %
completo: 0.025 %
completo: 0.02625 %
completo: 0.0275 %
completo: 0.02875 %
completo: 0.03 %
completo: 0.03125 %
completo: 0.0325 %
completo: 0.03375 %
completo: 0.035 %
completo: 0.03625 %
completo: 0.0375 %
completo: 0.03875 %
completo: 0.04 %
completo: 0.04125 %
completo: 0.0425 %
completo: 0.04375 %
completo: 0.045 %
completo: 0.04625 %
completo: 0.0475 %
completo: 0.04875 %
completo: 0.05 %
completo: 0.05125 %
completo: 0.0525 %
completo: 0.05375 %
completo: 0.055 %
completo: 0.05625 %
completo: 0.0575 %
completo: 0.05875 %
completo: 0.06 %
completo: 0.06125 %
completo: 0.0625 %
completo: 0.06375 %
completo: 0.065 %
compl

completo: 0.5375 %
completo: 0.53875 %
completo: 0.54 %
completo: 0.54125 %
completo: 0.5425 %
completo: 0.54375 %
completo: 0.545 %
completo: 0.54625 %
completo: 0.5475 %
completo: 0.54875 %
completo: 0.55 %
completo: 0.55125 %
completo: 0.5525 %
completo: 0.55375 %
completo: 0.555 %
completo: 0.55625 %
completo: 0.5575 %
completo: 0.55875 %
completo: 0.56 %
completo: 0.56125 %
completo: 0.5625 %
completo: 0.56375 %
completo: 0.565 %
completo: 0.56625 %
completo: 0.5675 %
completo: 0.56875 %
completo: 0.57 %
completo: 0.57125 %
completo: 0.5725 %
completo: 0.57375 %
completo: 0.575 %
completo: 0.57625 %
completo: 0.5775 %
completo: 0.57875 %
completo: 0.58 %
completo: 0.58125 %
completo: 0.5825 %
completo: 0.58375 %
completo: 0.585 %
completo: 0.58625 %
completo: 0.5875 %
completo: 0.58875 %
completo: 0.59 %
completo: 0.59125 %
completo: 0.5925 %
completo: 0.59375 %
completo: 0.595 %
completo: 0.59625 %
completo: 0.5975 %
completo: 0.59875 %
completo: 0.6 %
completo: 0.60125 %
complet